# 4. LSTM and GRU

<br>

## 강의 소개

- RNN을 개선한 알고리즘으로 등장했던 LSTM과 GRU에 대해서 다시 한번 살펴봅니다.
- LSTM과 GRU가 gradient flow를 개선할 수 있는 이유에 대해 조금 더 고민하는 시간이 됐으면 좋겠습니다.

<br>

## Further Reading

- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

<br>

## Further Question

- BPTT 이외에 RNN/LSTM/GRU의 구조를 유지하면서 gradient vanishing/exploding 문제를 완화할 수 있는 방법이 있을까요?
- RNN/LSTM/GRU 기반의 Language Model에서 초반 time step의 정보를 전달하기 어려운 점을 완화할 수 있는 방법이 있을까요?

<br>

## 4.1 Long Short-Term Memory (LSTM)

### 4.1.1 Core Idea

- pass cell state information straightly without any transformation
  - Solving long-term dependency problem
- LSTM 은 Vanilla RNN 이 갖고 있는 gradient vanishing/exploding 문제를 해결하고 time step 이 먼 경우에도 필요한 정보를 효과적으로 처리하고 학습할 수 있도록 개선한 모델이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1KtBkhf-NJSvJtYz6gqOT-aBxMBkYcYWc' width=800/>

- 출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

### 4.1.2 What is LSTM (Long Short-Term Memory)?

- RNN 이 가지고 있는 매 time step 마다 변하는 hidden state vector 를 단기 기억을 담당하는 기억 소자로 볼 수 있다.
- 이 단기 기억을 time step 이 진행됨에 따라서 보다 길게 기억할 수 있게 개선했다는 뜻에서 이름이 Long Short-Term Memory 로 지어졌다.

<br>

- RNN 에서 $h_t$ 를 계산하는 수식은 다음과 같았다.
  - $h_t = f_W \left( w_t, h_{t-1} \right)$
- LSTM 에서 input vector 이외에 2개의 벡터가 입력으로 들어오고 이 둘은 서로 다른 역할을 한다.
  - $C_t$
    - cell state vector
    - 위에서 들어오는 벡터
  - $h_t$
    - hidden state vector
    - 아래에서 들어오는 벡터
  - 이 두 가지 벡터가 각 time step 에서의 LSTM 모듈에서 계산이 되고 다음 time step 의 LSTM 모듈에 입력 벡터와 함께 입력으로 전달된다.
- LSTM 모듈에서 계산되는 수식을 다음과 같이 표현할 수 있다.
  - $\{C_t, h_t\} = LSTM(x_t, C_{t-1}, h_{t-1})$

<br>

- $C_t$ 에서 우리가 필요로 하는 정보들을 갖고 있다.
- $h_t$ 는 cell state vector 를 한번 더 가공해서 그 time step 에서 노출할 필요가 있는 정보만을 남긴, 즉 필터링된 정보들을 갖고 있다.
- $h_t$ 는 output layer 의 입력 벡터로도 사용된다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1KtBkhf-NJSvJtYz6gqOT-aBxMBkYcYWc' width=800/>

- 출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

### 4.1.3 Long short-term memory 연산 과정

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ybYSvCwGJllt2lH5HaIBKrmkNOVHoZaN' width=800/>

- 출처: Long short-term memory, Neural computation’97

<br>

위 그림의 왼쪽 부분

- 먼저 입력으로 들어오는 벡터 중 $x_t$ 와 $h_{t-1}$ 벡터들을 선형변환을 통해 새로운 벡터로 만들어낸다.
  - $x_t$ 와 $h_{t-1}$ 벡터의 차원을 각각 `h` 라고 한다면 이 두 벡터를 concatenate 한 벡터의 차원은 `2h` 가 된다.
  - 총 4개로 분할될 수 있는 벡터를 만들어주는 선형변환을 해줘야 하기 때문에 가중치 행렬의 차원은 `4hx2h` 가 된다.

<br>

위 그림의 중간 부분

- 그런 다음 이렇게 생긴 벡터를 4개로 쪼갠 다음에 쪼개진 각각의 벡터들에 대해 3개의 벡터는 sigmoid 함수를 적용하고 나머지 하나의 벡터는 tanh 함수를 적용하여 총 4개의 벡터들을 생성하게 된다.
  - sigmoid 함수를 적용하는 벡터들은 0 ~ 1 사이의 값을 갖게 되고, 다른 벡터와의 element wise multiplication 을 통해서 벡터가 갖고 있는 값에 곱해줌으로서 **원래 값이 가지고 있는 정보 중 일부만을 가지도록 하는 역할**을 한다.
  - tanh 함수를 적용하는 벡터는 -1 ~ 1 사이의 값을 갖게 되고, RNN 에서의 tanh 함수와 같이 LSTM 에서 계산되는 정보들을 변환해주는 작업이라고 볼 수 있다.
- 이 벡터들은 아래와 같은 이름의 gate 로 불린다.
  - `i` : **Input gate**, Whether to write to cell
  - `f` : **Forget gate**, Whether to erase cell
  - `o` : **Output gate**, How much to reveal cell
  - `g` : **Gate gate**, How much to write to cell
- 이 벡터들은 cell state vector 를 계산하기 위한 중간 결과물로서 사용된다.

<br>

### 4.1.4 4개의 gate 의 역할

- A gate exists for controlling how much information could flow from cell state
- i, f, o, g 벡터들의 역할을 자세히 살펴보자.
- 기본적으로 이 벡터들은 전 time step 에서 넘어온 $C_{t-1}$ 벡터를 적절하게 변환하는 데 사용된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kkVIe7NWQnkl-x0htR2QtkcfOwA6TEG4' width=800/>

- 출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

#### 4.1.4.1 Forget gate

- $f_{t}=\sigma\left(W_{f} \cdot\left[h_{t-1}, x_{t}\right]+b_{f}\right)$

<br>

- $C_{t-1}$ 이 3차원 벡터(`[3, 5, -2]`)라고 할 때 가장 먼저 forget gate $f_t$ 와 곱해지게 된다.
- $f_t$ 의 값이 `[0.7, 0.4, 0.8]` 로 계산된다면 `[3, 5, -2]` 와 element wise 로 곱해져서 `[2.1, 2.0, -1.6]` 이 된다.
- 이는 forget gate 에서 이전 time step 에서 넘어온 cell state vector 의 정보 중 각각 70%, 40%, 80% 만을 유지하겠다는 것을 의미하게 된다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1-C--B1MmboS5rmXWMB3qR5bmZe-QngV1' width=600/>

- 출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

#### 4.1.4.2 Input gate & Gate gate

- Generate information to be added and cut it by input gate
  - $\widetilde{C}_{t}$ 를 $C_{t-1}$ 에 바로 더해주지 않고 $i_t$ 를 곱해서 더해주는 이유는 더해주고자 하는 정보 중 일부를 덜어주고 더해주는 것으로 이해할 수 있다.
  - input gate: $i_{t}=\sigma\left(W_{i} \cdot\left[h_{t-1}, x_{t}\right]+b_{i}\right)$
  - gate gate: $\widetilde{C}_{t}=\tanh \left(W_{C} \cdot\left[h_{t-1}, x_{t}\right]+b_{C}\right)$
- Generate new cell state by adding current information to previous cell state
  - input gate 와 gate gate 를 같은 dimension 끼리 곱해준 값을 forget gate 가 곱해진 $C_{t-1}$ 에 더해줌으로서 현재 time step 에서의 $C_t$ 를 만들어낸다. (원하는 정보를 더해준다라고 볼 수 있다.)
  - $C_{t}=f_{t} \cdot C_{t-1}+i_{t} \cdot \widetilde{C}_{t}$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=18KYxbBdBdHaCPSE8TkyjejREEMk-K1zJ' width=600/>

- 출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

#### 4.1.4.3 Output gate

- Generate hidden state by passing cell state to tanh and output gate
- Pass this hidden state to next time step, and output or next layer if needed
  - $o_{t}=\sigma\left(W_{o}\left[h_{t-1}, x_{t}\right]+b_{o}\right)$
  - $h_{t}=o_{t} \cdot \tanh \left(C_{t}\right)$

<br>

- hidden state vector $h_t$ 를 만들어줘야 한다.
- $C_t$ 에 tanh 를 적용해서 -1~1 사이의 범위를 갖는 벡터로 만들어준 다음 output gate $o_t$ 의 값을 element wise 로 곱해줌으로서 $h_t$ 를 만든다.
- $o_t$ 또한 sigmoid 함수가 적용되어 0 ~ 1 사이의 값을 가지므로 cell state 가 가지는 정보 중에서 일부의 정보만을 전달하는 것으로 볼 수 있다.
- 이러한 것을 보면 $C_t$ 는 모든 정보를 갖고 있지만 $h_t$ 는 현재 time step 에서 예측값을 만드는 output layer 의 입력으로 사용된다는 점에서 해당 time step 에서 예측값을 생성하는데 필요한 정보만을 필터링하여 담은 것으로 볼 수 있다.
- $h_t$ 는 다음 time step 으로 넘어감과 동시에 위로 올라가는 모습을 볼 수 있는데 이것이 바로 현재 time step에서 예측을 수행할 때 output layer 의 입력으로 사용되기 위해 전달되는 값이다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1DlcrHXiRC8IqBBeqrUFRC0yny_j6gEH_' width=600/>

- 출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

## 4.2 Gated Recurrent Unit (GRU)

### 4.2.1 What is GRU

- GRU 는 LSTM 을 경량화해서 적은 메모리 요구량과 빠른 계산 시간이 가능하도록 만든 모델이다.
- GRU 의 가장 큰 특징으로서 **LSTM 에서 2가지 형태의 벡터로 존재하던 cell state vector 와 hidden state vector 를 일원화해서 hidden state vector $h_t$ 만이 존재한다는 것**이 있다.
  - 일원화된 hidden state 는 LSTM 의 cell state vector 와 유사한 역할을 하게 된다.
- 또다른 특징으로는 **LSTM 에서는 2개의 gate(forget, input)로 하던 연산을 하나의 gate를 통해 수행했다는 점**이 있다.
  - LSTM 에서 $C_t$ 를 만들기 위해 forget gate $f_t$ 와 input gate $i_t$ 를 사용했던 것과 달리 GRU 에서는 input gate $z_t$ 만을 사용한다.
  - forget gate 자리에는 input gate $z_t$ 를 1에서 뺀 $1-z_t$ 를 사용한다.
    - input gate 값이 커지면 forget gate 의 값은 작아진다.
  - hidden state vector $h_t$ 는 이전 time step 의 hidden state vector $h_{t-1}$ 과 현재 만들어진 정보인 $\tilde{h}_{t}$ 의 정보간에 독립적인 gate 를 한 후 더하는 것이 아니라 두 정보간의 가중 평균을 내는 것으로 이해할 수 있다.
- 그렇지만 GRU 의 전체적인 동작 원리는 LSTM 과 굉장히 유사하다.

<br>

- $z_{t}=\sigma\left(W_{z} \cdot\left[h_{t-1}, x_{t}\right]\right)$
- $r_{t}=\sigma\left(W_{r} \cdot\left[h_{t-1}, x_{t}\right]\right)$
- $\tilde{h}_{t}=\tanh \left(W \cdot\left[r_{t} \cdot h_{t-1}, x_{t}\right]\right)$
- $h_{t}=\left(1-z_{t}\right) \cdot h_{t-1}+z_{t} \cdot \widetilde{h}_{t}$
- c.f) $C_{t}=f_{t} \cdot C_{t-1}+i_{t} \cdot \widetilde{C}_{t}$ in LSTM

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sm4LHadyV6Y6CART6FdgZnB7Sz70qCd-' width=600/>

- 출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

## 4.3 Backpropagation in LSTM, GRU

- Uninterrupted(연속된) gradient flow!
- 주요한 정보를 담고 있는 cell state vector 가 업데이트되는 과정이 original RNN 에서 $W_{hh}$ 를 거듭해서 곱해주는 것과 같은 연산이 아니라 전 time step 의 cell state vector 에서 그때 그때 forget gate 를 곱하고 필요한 정보를 곱셈이 아닌 덧셈을 통해서 원하는 정보를 만들어준다는 사실로 인해서 gradient vanishing/exploding 문제가 사라지게 된다.
  - 덧셈 연산은 backpropagation 을 수행할 때 gradient 를 복사해주는 역할을 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1-S9VngW0R_OAZwr4iF0cwBGWXnPHb7ZX' width=900/>

- 출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

## 4.4 Summary on RNN/LSTM/GRU

- RNNs allow a lot of <font color='red'>flexibility</font> in architecture design
  - RNN 은 다양한 길이를 가질 수 있는 sequence data 에 특화된 형태의 딥러닝 모델 구조이다.
- Vanilla RNNs are <font color='red'>simple</font> but don't work very well
- Backward flow of gradients in RNN can <font color='red'>explode or vanish</font>
  - Vanilla RNN 은 구조가 매우 간단하지만 학습 시 gradient vanishing/exploding 문제가 발생할 수 있어 많이 사용되지 않는다.
- Common to use LSTM or GRU
  - their additive interactions <font color='red'>improve gradient flow</font>
  - LSTM 과 GRU는 cell state 를 업데이트하는 방식이 **덧셈**에 기반하기 때문에 gradient vanishing/exploding 문제를 피할 수 있다.

<br>

## 4.5 실습: LSTM, GRU

1. 기존 RNN 과 다른 부분에 대해서 배운다.
2. 이전 실습에 이어 다양한 적용법을 배운다.

<br>

### 4.5.1 필요 패키지 import

In [2]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch

<br>

### 4.5.2 데이터 전처리

- 아래의 sample data 를 확인해보자. (이전 실습과 동일)

In [3]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [4]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 35335.33it/s]


In [5]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

<br>

### 4.5.3 LSTM 사용

- LSTM 에선 cell state 가 추가된다.
- cell state 의 shape 은 hidden state 의 shape 과 동일하다.

In [6]:
embedding_size = 256
hidden_size = 512
num_layers = 1
num_dirs = 1

embedding = nn.Embedding(vocab_size, embedding_size)
lstm = nn.LSTM(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size)) # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size)) # (num_layers * num_dirs, B, d_h)

In [7]:
# d_w: word embedding size
batch_emb = embedding(batch) # (B, L, d_w)
batch_emb.shape

torch.Size([10, 20, 256])

In [12]:
packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, (h_n, c_n) = lstm(packed_batch, (h_0, c_0))
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)
print(c_n.shape)

PackedSequence(data=tensor([[ 0.0349, -0.0717, -0.0494,  ..., -0.1301,  0.0156, -0.0285],
        [ 0.0039, -0.0848, -0.1199,  ..., -0.0234,  0.1441,  0.0354],
        [ 0.0924,  0.0439, -0.0758,  ...,  0.0022,  0.0554,  0.0257],
        ...,
        [-0.1049,  0.0285,  0.1007,  ..., -0.1961, -0.0256,  0.0783],
        [-0.0058, -0.0997, -0.0244,  ...,  0.1517,  0.2395,  0.0274],
        [-0.0137,  0.1824, -0.2192,  ...,  0.1396,  0.1369,  0.0427]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])


In [13]:
outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)
print(output_lens)

torch.Size([20, 10, 512])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


<br>

### 4.5.4 GRU 사용

- GRU 는 cell state 가 없어 RNN 과 동일하게 사용 가능하다.
- GRU 를 이용하여 Language Model task 를 수행해보자.

In [37]:
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [38]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [39]:
input_id = batch.transpose(0, 1)[0, :] # B
input_id

tensor([85, 58, 87, 22, 70, 19, 93, 94, 80, 62])

In [40]:
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

<br>

- Teacher forcing 없이 이전에 얻은 결과를 다음 input 으로 이용한다.

In [41]:
for t in range(max_len):
    input_emb = embedding(input_id).unsqueeze(0) # (1, B, d_w)
    output, hidden = gru(input_emb, hidden) # output: (1, B, d_h), hidden: (1, B, d_h)

    # V: vocab size
    output = output_layer(output) # (1, B, V)
    probs, top_id = torch.max(output, dim=-1) # probs: (1, B), top_id: (1, B)

    print("*" * 50)
    print(f"Time step: {t}")
    print(output.shape)
    print(probs.shape)
    print(top_id.shape)

    input_id = top_id.squeeze(0)  # (B)

**************************************************
Time step: 0
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 1
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 2
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 3
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 4
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 5
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 6
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 7
torch.Size([1, 10, 100])
torch.Si

<br>

- `max_len` 만큼의 for문을 돌면서 모든 결과물의 모양을 확인했지만 만약 종료 조건(ex. 문장의 끝을 나타내는 end token 등)이 되면 중간에 생성을 그만둘수도 있다.

<br>

### 4.5.5 양방향 및 여러 layer 사용

- 이번엔 양방향 + 2개 이상의 layer 를 쓸 때 얻을 수 있는 결과에 대해 알아보자.

In [42]:
num_layers = 2
num_dirs = 2
dropout = 1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

<br>

- Bidirectional 이 되었고 layer 의 개수가 2로 늘었기 때문에 hidden state 의 shape 도 `(4, B, d_h)` 가 된다.

In [43]:
# d_w: word embedding size
# num_layers: layer 의 개수
# num_dirs: 방향의 개수

batch_emb = embedding(batch) # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size)) # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

PackedSequence(data=tensor([[ 0.0406, -0.0119, -0.0071,  ..., -0.2520, -0.1993, -0.1060],
        [ 0.0410,  0.1082, -0.0604,  ..., -0.0563,  0.1071, -0.2233],
        [-0.0133,  0.1561, -0.0856,  ...,  0.2162,  0.0425, -0.0397],
        ...,
        [ 0.0271,  0.0700,  0.1165,  ...,  0.1129, -0.0277,  0.0016],
        [-0.1314, -0.0266,  0.0435,  ..., -0.1107,  0.0446, -0.0148],
        [-0.1458, -0.0240, -0.0314,  ..., -0.0883,  0.0723, -0.0078]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 1024])
torch.Size([4, 10, 512])


In [44]:
outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

torch.Size([20, 10, 1024])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


<br>

- 각각의 결과물의 shape는 다음과 같습니다.
  - `outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
  - `h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [45]:
batch_size = h_n.shape[1]
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)

tensor([[[[-3.5023e-01, -1.1334e-01, -1.6157e-01,  ...,  4.5304e-01,
            1.8469e-01, -2.1680e-02],
          [-5.7206e-02,  1.3418e-01,  1.2187e-01,  ..., -2.0633e-01,
            1.1726e-01, -1.4778e-01],
          [ 7.1233e-02, -3.1554e-01,  1.7080e-01,  ..., -1.2315e-01,
           -1.9801e-01, -9.2919e-02],
          ...,
          [ 3.1512e-01, -4.1218e-01,  1.7909e-01,  ...,  3.0241e-01,
            4.4074e-01, -1.5008e-01],
          [ 7.7649e-02, -9.8056e-03, -4.0156e-01,  ..., -3.6668e-01,
           -2.1716e-01,  2.9913e-02],
          [ 3.1635e-01,  1.4461e-01,  2.2485e-02,  ..., -2.8990e-01,
            2.8460e-01,  1.2556e-01]],

         [[-1.3612e-01,  4.7373e-01, -2.7130e-01,  ..., -8.1935e-02,
            6.3042e-02, -1.7000e-01],
          [-1.3340e-01, -8.0582e-02, -4.4041e-02,  ...,  9.3998e-02,
            1.4388e-02, -4.7930e-01],
          [-4.2317e-01,  2.9652e-03,  2.6890e-01,  ...,  5.4826e-01,
           -5.1891e-02,  1.4421e-01],
          ...,
     